### IBM Data Science Certificat Professionnel

## Applied Data Science Capstone

# The Battle of the Neighbourhoods of Toronto

# 1. Introduction

## Description of problem and Background

Leaving your city to settle in another city like Tornonto, requires planning and good information on the city in which you want to live with your family. Perhaps you would like to have additional information so quickly regarding districts such as :
<ul>
    <li>The average selling price of houses by district.</li>
    <li>The rate of the most common crimes by district to take care of your security </li>
    <li>The average incomes by district to improve your savings and take care of your family </li>
    <li>How similar certain neighborhoods are to plan your move.</li>

<ul>


## Expected / Interested Audience


Toronto, capital of the province of Ontario, is one of the major Canadian cities, located on the northwest shore of Lake Ontario. This dynamic metropolis includes a number of dizzying skyscrapers, dominated by the iconic CN tower. Toronto is also home to many green spaces, from Queens Park Oval Park to High Park, which covers an area of ​​1.6 km2 and offers trails, sports facilities and a zoo.  
This project is dedicated to all those who want to come and live in Toronto and who want to know:
* How much does it cost  a house in the neighbourhood of Toronto regarding the number of bedrooms and bathrooms,the square footage,the type ,the average incomes in this district etc… ?
* How does the average most common crime rate vary in relation to neighborhoods ?
* How similar certain neighborhoods are to plan your move.



# 2. Data 

## 2.1. Data collections


To resolve this problem we are going to use  :
1. [Toronto Housing  Info Dataset](https://github.com/MarvelAmazon/couresa-ibm-capstone-project/blob/master/dataset/houses.csv)- Obtained from residences listing websites through web scraping methods by [slavaspirin](https://github.com/slavaspirin) that contains:
    <ul>
    <li>Addresses of houses</li>
    <li>Number of bedrooms for each house</li>
    <li>Number of bathrooms for each house</li>
    <li>Latitudes</li>
    <li>Longitudes</li>
    <li>The names of Neighborhood</li>
    <li>The type of the residences: House/Townhouse/Condo</li>
    <li>Square footage of the house</li>
    <li>Listing prices</li>
    <li>Final prices etc...</li>
    </ul>
2. [MCI Dataset](http://data.torontopolice.on.ca/datasets/mci-2014-to-2018)  - It  includes all Major Crime Indicators (MCI) 2014 to 2018 occurrences by reported date and related offences to predict crime on Toronto Neighbourhoods such as:
    * Occurrence years
    * Occurrence months
    * Occurrence days
    * Occurrence days of year
    * Occurrence days of week
    * Occurrence hours
    * Major Crime Indicators (MCI)
    * Neighbourhood.
3. [Toronto Neighbourhoods Shapefile Dataset](https://github.com/MarvelAmazon/couresa-ibm-capstone-project/blob/master/dataset/Neighbourhoods.shp) - It contains geodata to perform great data visualization and clustering analysis.
    * Latitudes
    * Longitudes
    * The names of Neighborhood
    * Geometries

4. Venues Data Dataset - Obtained through [Foursquare API](https://developer.foursquare.com/) Numbers of Venues closed to the Neighborhood The frequency of each Venue Category like Place, Coffee, Chinese Restaurant, Italian Restaurant, etc.  Office, Bus Stop, Pizza Place, Coffee.




## 2.2 Data acquisition and Libraries used


In [18]:
import warnings
warnings.filterwarnings('ignore') 
#import requests # HTTP library
import requests
import pandas as pd # for data analysis
import numpy as np  # data in a vectorized manner manipulation
# Matplotlib and associated plotting modules for visualization
import matplotlib.cm as cm
import matplotlib.colors as colors

import matplotlib.pyplot as plt

import statsmodels.api as sm # implement statistic tests
import time # use time
from geopy.geocoders import Nominatim  # for geocoders referencing
import geopandas as gpd # for spatial dataset
import seaborn as sns # for plotting and visulalization
from scipy import stats # statistic computation
from bs4 import BeautifulSoup # scrapping web site
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re # regular expression
import folium
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor,ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
import xgboost as xgb
from sklearn.metrics import r2_score,mean_squared_error,explained_variance_score,mean_absolute_error
from sklearn.pipeline import Pipeline
from  sklearn.linear_model import LinearRegression,Ridge
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
# import k-means from clustering stage
from sklearn.cluster import KMeans
sns.set_style("white")

## 2.3 Data acquisition look and feel

### 2.3.1 Toronto Housing  Info Dataset

In [19]:
house_data = pd.read_csv('./dataset/houses.csv')
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15236 entries, 0 to 15235
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 15236 non-null  int64  
 1   title                 15236 non-null  object 
 2   final_price           15236 non-null  int64  
 3   list_price            15236 non-null  int64  
 4   bedrooms              15236 non-null  object 
 5   bathrooms             15236 non-null  object 
 6   sqft                  15236 non-null  object 
 7   parking               15236 non-null  object 
 8   description           15187 non-null  object 
 9   mls                   15236 non-null  object 
 10  type                  15236 non-null  object 
 11  full_link             15236 non-null  object 
 12  full_address          15236 non-null  object 
 13  lat                   15236 non-null  float64
 14  long                  15236 non-null  float64
 15  city_district      

### 2.3.2 MCI Dataset

In [20]:
mci_2014_2018 = pd.read_csv('./dataset/mci_2014_to_2018.csv')
mci_2014_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167525 entries, 0 to 167524
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   X                    167525 non-null  float64
 1   Y                    167525 non-null  float64
 2   Index_               167525 non-null  int64  
 3   event_unique_id      167525 non-null  object 
 4   occurrencedate       167525 non-null  object 
 5   reporteddate         167525 non-null  object 
 6   premisetype          167525 non-null  object 
 7   ucr_code             167525 non-null  int64  
 8   ucr_ext              167525 non-null  int64  
 9   offence              167525 non-null  object 
 10  reportedyear         167525 non-null  int64  
 11  reportedmonth        167525 non-null  object 
 12  reportedday          167525 non-null  int64  
 13  reporteddayofyear    167525 non-null  int64  
 14  reporteddayofweek    167525 non-null  object 
 15  reportedhour     

### 2.3.3 Toronto Neighbourhoods Shapefile Dataset

In [21]:
# 1. retreives geodata from Neighbourhoods.shp shapefile
data_geom = gpd.read_file('./dataset/Neighbourhoods.shp')
display(data_geom.head())
# 2. selects columns needed to perform operations
data_geom= data_geom[['FIELD_5','FIELD_11','FIELD_12','geometry','FIELD_7']]
# 3. renames columns 'FIELD_11' to 'x','FIELD_12' to 'y','FIELD_5' to 'district_code'
data_geom.rename(columns ={'FIELD_11':'x','FIELD_12': 'y','FIELD_5':'district_code','FIELD_7':'city_district'},inplace =True)
data_geometry_final = data_geom
display(data_geometry_final.info())
display(data_geometry_final.head())

,FIELD_1,FIELD_2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_7,FIELD_8,FIELD_9,FIELD_10,FIELD_11,FIELD_12,FIELD_13,FIELD_14,FIELD_15,geometry
0,141,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((309967.120 4837586.769, 310047.783 4..."
1,142,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((311976.621 4840247.041, 312084.257 4..."
2,143,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"POLYGON ((313572.832 4837693.591, 313555.868 4..."
3,144,25886593,25926665,49885,27,27,York University Heights (27),York University Heights (27),None,None,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"POLYGON ((304374.200 4846441.252, 304406.841 4..."
4,145,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),None,None,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"POLYGON ((309660.957 4840414.596, 309626.870 4..."


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   district_code  140 non-null    int64   
 1   x              140 non-null    float64 
 2   y              140 non-null    float64 
 3   geometry       140 non-null    geometry
 4   city_district  140 non-null    object  
dtypes: float64(2), geometry(1), int64(1), object(1)
memory usage: 5.6+ KB


None

,district_code,x,y,geometry,city_district
0,94,-79.425515,43.676919,"POLYGON ((309967.120 4837586.769, 310047.783 4...",Wychwood (94)
1,100,-79.403590,43.704689,"POLYGON ((311976.621 4840247.041, 312084.257 4...",Yonge-Eglinton (100)
2,97,-79.397871,43.687859,"POLYGON ((313572.832 4837693.591, 313555.868 4...",Yonge-St.Clair (97)
3,27,-79.488883,43.765736,"POLYGON ((304374.200 4846441.252, 304406.841 4...",York University Heights (27)
4,31,-79.457108,43.714672,"POLYGON ((309660.957 4840414.596, 309626.870 4...",Yorkdale-Glen Park (31)


### 2.3.4 Toronto Neighbourhoods  venues

In [22]:
# @hidden_cell

CLIENT_ID = 'RVJDHM5QFQZ1FLRTQ2G0VHPUZES0YMPJVDBDBHHOTCPGTBN4' # your Foursquare ID
CLIENT_SECRET = '0ONU0KYNRFZG5HXX1FGZWAHD1K5YRYDAKNSAB2L1UWAM0QXQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [24]:
toronto_latitude = data_geometry_final.loc[30, 'y'] # neighborhood latitude value
toronto_longitude = data_geometry_final.loc[30, 'x'] # neighborhood longitude value

toronto_name = data_geometry_final.loc[0, 'city_district'] # nteighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(toronto_name, 
                                                               toronto_latitude, 
                                                               toronto_longitude))

Latitude and longitude values of Wychwood (94) are 43.6974081338, -79.2797083433.


In [25]:
# @hidden_cell
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 20000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    toronto_latitude,
    toronto_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RVJDHM5QFQZ1FLRTQ2G0VHPUZES0YMPJVDBDBHHOTCPGTBN4&client_secret=0ONU0KYNRFZG5HXX1FGZWAHD1K5YRYDAKNSAB2L1UWAM0QXQ&v=20180605&ll=43.6974081338,-79.2797083433&radius=20000&limit=100'

In [26]:
# function that defines the most common revenues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e6fb9cc98205d001be87cee'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 238,
  'suggestedBounds': {'ne': {'lat': 43.87740831380018,
    'lng': -79.03120973046795},
   'sw': {'lat': 43.517407953799825, 'lng': -79.52820695613205}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a520c4f720e3',
       'name': 'The Fox Theatre',
       'location': {'address': '2236 Queen St East',
        'crossStreet': 'at Beech Ave',
        'lat': 43.67280115048827,
        'lng': -79.28727198292374,
        'labeledLatLngs': [{'label': 'display',

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
# convert venues to dataset
toronto_venues = getNearbyVenues(names=data_geometry_final['district_code'],
                                   latitudes=data_geometry_final['y'],
                                   longitudes=data_geometry_final['x']
                                  )

94
100
97
27
31
114
38
105
103
56
84
19
132
29
12
130
17
135
73
115
2
99
104
18
50
36
82
68
74
54
121
107
58
80
45
23
67
46
10
72
4
111
86
98
131
89
28
139
85
70
40
116
16
118
61
63
3
55
81
79
43
77
136
1
35
113
91
119
51
37
7
137
64
60
129
128
20
95
42
34
76
52
49
39
112
127
122
24
69
108
41
57
30
71
109
96
133
75
120
33
123
92
66
59
47
126
93
26
83
62
9
138
5
32
11
13
44
102
101
25
65
140
53
88
87
134
48
8
21
22
106
125
14
90
110
124
78
6
15
117


In [31]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
1,13,13,13,13,13,13
2,4,4,4,4,4,4
3,16,16,16,16,16,16
4,2,2,2,2,2,2
5,5,5,5,5,5,5
6,4,4,4,4,4,4
7,1,1,1,1,1,1
8,4,4,4,4,4,4
9,3,3,3,3,3,3


In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 286 uniques categories.


In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)
toronto_onehot.head()

,Neighbourhood,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped = toronto_grouped.rename(columns={'Neighbourhood': 'district_code'})
display(toronto_grouped.head())
toronto_grouped.to_csv('./dataset/neighbourhood_values.csv')

,district_code,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store

In [35]:
# that function return all must commons venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
toronto_grouped = pd.read_csv('./dataset/neighbourhood_values.csv')
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['district_code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['district_code'] = toronto_grouped['district_code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,district_code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,district_code,Hotel,Department Store,Racecourse,Bank,Coffee Shop,Gym / Fitness Center,Paper / Office Supplies Store,Swiss Restaurant,Mediterranean Restaurant
1,2,district_code,Convenience Store,Park,Coffee Shop,Japanese Restaurant,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
2,3,district_code,Indian Restaurant,Caribbean Restaurant,Spa,Bank,Coffee Shop,Supermarket,Asian Restaurant,Thai Restaurant,Pizza Place
3,4,district_code,Flower Shop,Playground,Fast Food Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
4,5,district_code,Hakka Restaurant,Construction & Landscaping,Business Service,Pool,African Restaurant,Hockey Field,Farmers Market,Electronics Store,Empanada Restaurant


In [37]:
neighborhoods_venues_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   district_code           138 non-null    int64 
 1   1st Most Common Venue   138 non-null    object
 2   2nd Most Common Venue   138 non-null    object
 3   3rd Most Common Venue   138 non-null    object
 4   4th Most Common Venue   138 non-null    object
 5   5th Most Common Venue   138 non-null    object
 6   6th Most Common Venue   138 non-null    object
 7   7th Most Common Venue   138 non-null    object
 8   8th Most Common Venue   138 non-null    object
 9   9th Most Common Venue   138 non-null    object
 10  10th Most Common Venue  138 non-null    object
dtypes: int64(1), object(10)
memory usage: 12.0+ KB


In [38]:
toronto_grouped = toronto_grouped.drop(columns=['Unnamed: 0'],axis =0)
final_dataset = pd.merge(data_geometry_final,toronto_grouped,how='left',on='district_code')
display(final_dataset.info())
display(final_dataset.head())

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 140 entries, 0 to 139
Columns: 290 entries, district_code to Zoo Exhibit
dtypes: float64(287), geometry(1), int64(1), object(1)
memory usage: 318.3+ KB


None

,district_code,x,y,geometry,city_district,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,V

## 2.3 Data cleaning

### 2.3.1 Cleaning the house dataset

In [54]:
house_data = pd.read_csv('./dataset/houses.csv')
# filter column sqft with value N/A sq. ft.
sqft_na_value = house_data[house_data.sqft == 'N/A sq. ft.']
display(sqft_na_value.shape)
display('percent of missing data column sqft (% {})'.format(round((sqft_na_value.shape[0]/house_data.shape[0])*100,2)))

(4522, 18)

'percent of missing data column sqft (% 29.68)'

In [55]:
# sqft (% 29.68)  > 10 
house_data = house_data[house_data.sqft != 'N/A sq. ft.']
house_data.shape

(10714, 18)

In [56]:
# convert bedrooms string to numerical data
def conver_string_num(x):
    x = str(x)
    x = x.replace(' ','')
    result = 0
    tampon = x.split('+')
    if len(tampon) == 1:
        val = re.sub('\D','',tampon[0])
        if val !='':
            result = np.float(re.sub('\D','',tampon[0]))
    elif len(tampon) >1:
        digit_1 = float(tampon[0])
        digit_2 = np.float(re.sub('\D','',tampon[1]))
        result = digit_1+digit_2
    return result 

In [57]:
# extracts numerical infos from [bedrooms,bathrooms,parking]
house_data['bedrooms_num'] = house_data['bedrooms'].apply(lambda x:conver_string_num(x))
house_data['bathrooms_num'] = house_data['bathrooms'].apply(lambda x:conver_string_num(x))
house_data['parking_num'] = house_data['parking'].apply(lambda x:conver_string_num(x))

In [58]:
# converts final_price  and mean_district_income to 10k dollars
house_data['final_price_10k'] = house_data['final_price']/10000.0
house_data['mean_district_income_10k'] = np.round(house_data['mean_district_income']/10000.0,2)


In [59]:
## remove (-) in city_district columns
house_data.city_district =house_data.city_district.str.replace('-',' ')


In [60]:
# select feautres to improve data analysis 
selected_features =['lat','long','type','city_district','district_code','sqft','parking','bedrooms_num','bathrooms_num','final_price_10k','mean_district_income_10k']
house_final_data =house_data[selected_features]
display(house_final_data.info())
display(house_final_data.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10714 entries, 0 to 15234
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lat                       10714 non-null  float64
 1   long                      10714 non-null  float64
 2   type                      10714 non-null  object 
 3   city_district             10714 non-null  object 
 4   district_code             10714 non-null  int64  
 5   sqft                      10714 non-null  object 
 6   parking                   10714 non-null  object 
 7   bedrooms_num              10714 non-null  float64
 8   bathrooms_num             10714 non-null  float64
 9   final_price_10k           10714 non-null  float64
 10  mean_district_income_10k  10714 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1004.4+ KB


None

,lat,long,type,city_district,district_code,sqft,parking,bedrooms_num,bathrooms_num,final_price_10k,mean_district_income_10k
0,43.661896,-79.385748,Condo Apt,Bay Street Corridor,76,800–899 sq. ft.,1 parking,3.0,2.0,85.5,5.65
2,43.708472,-79.397498,Condo Apt,Mount Pleasant West,104,500–599 sq. ft.,no parking,1.0,1.0,55.0,5.70
3,43.638399,-79.414448,Condo Apt,Niagara,82,600–699 sq. ft.,1 parking,2.0,1.0,66.5,7.06
5,43.601518,-79.510062,Condo Apt,New Toronto,18,0-499 sq. ft.,1 parking,0.0,1.0,36.8,4.41
6,43.714068,-79.403338,Detached,Lawrence Park South,103,2500–3000 sq. ft.,2 parking,4.0,5.0,270.0,16.92


### 2.2.2 mci_2014_2018 Data cleaning

In [63]:
mci_2014_2018 = pd.read_csv('./dataset/mci_2014_to_2018.csv')
display(mci_2014_2018.info())
display(mci_2014_2018.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167525 entries, 0 to 167524
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   X                    167525 non-null  float64
 1   Y                    167525 non-null  float64
 2   Index_               167525 non-null  int64  
 3   event_unique_id      167525 non-null  object 
 4   occurrencedate       167525 non-null  object 
 5   reporteddate         167525 non-null  object 
 6   premisetype          167525 non-null  object 
 7   ucr_code             167525 non-null  int64  
 8   ucr_ext              167525 non-null  int64  
 9   offence              167525 non-null  object 
 10  reportedyear         167525 non-null  int64  
 11  reportedmonth        167525 non-null  object 
 12  reportedday          167525 non-null  int64  
 13  reporteddayofyear    167525 non-null  int64  
 14  reporteddayofweek    167525 non-null  object 
 15  reportedhour     

None

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.385193,43.659229,2349,GO-20149004286,2014-06-20T10:55:00.000Z,2014-06-20T13:20:00.000Z,Apartment,2130,210,Theft Over,2014,June,20,171,Friday,13,2014.0,June,20.0,171.0,Friday,10,Theft Over,D52,76,Bay Street Corridor (76),43.659229,-79.385193,2001
1,-79.425400,43.777592,2350,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1457,100,Pointing A Firearm,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Assault,D32,36,Newtonbrook West (36),43.777592,-79.425400,2002
2,-79.425400,43.777592,2351,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1610,100,Robbery With Weapon,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Robbery,D32,36,Newtonbrook West (36),43.777592,-79.425400,2003
3,-79.210373,43.801727,2352,GO-20142412127,2014-07-02T01:30:00.000Z,2014-07-02T05:40:00.000Z,House,2120,200,B&E,2014,July,2,183,Wednesday,5,2014.0,July,2.0,183.0,Wednesday,1,Break and Enter,D42,132,Malvern (132),43.801727,-79.210373,2004
4,-79.254334,43.835884,2354,GO-20142417548,2014-07-02T20:52:00.000Z,2014-07-02T20:57:00.000Z,Commercial,1430,100,Assault,2014,July,2,183,Wednesday,20,2014.0,July,2.0,183.0,Wednesday,20,Assault,D42,130,Milliken (130),43.835884,-79.254334,2005


In [64]:
mci_2014_2018 = pd.read_csv('./dataset/mci_2014_to_2018.csv')
print(mci_2014_2018.shape)
mci_2014_2018['Neighbourhood'] = mci_2014_2018.Neighbourhood.apply(lambda x: x.split('(')[0].replace('-',' '))
mci_2014_2018.head()

(167525, 29)


,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.385193,43.659229,2349,GO-20149004286,2014-06-20T10:55:00.000Z,2014-06-20T13:20:00.000Z,Apartment,2130,210,Theft Over,2014,June,20,171,Friday,13,2014.0,June,20.0,171.0,Friday,10,Theft Over,D52,76,Bay Street Corridor,43.659229,-79.385193,2001
1,-79.425400,43.777592,2350,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1457,100,Pointing A Firearm,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Assault,D32,36,Newtonbrook West,43.777592,-79.425400,2002
2,-79.425400,43.777592,2351,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1610,100,Robbery With Weapon,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Robbery,D32,36,Newtonbrook West,43.777592,-79.425400,2003
3,-79.210373,43.801727,2352,GO-20142412127,2014-07-02T01:30:00.000Z,2014-07-02T05:40:00.000Z,House,2120,200,B&E,2014,July,2,183,Wednesday,5,2014.0,July,2.0,183.0,Wednesday,1,Break and Enter,D42,132,Malvern,43.801727,-79.210373,2004
4,-79.254334,43.835884,2354,GO-20142417548,2014-07-02T20:52:00.000Z,2014-07-02T20:57:00.000Z,Commercial,1430,100,Assault,2014,July,2,183,Wednesday,20,2014.0,July,2.0,183.0,Wednesday,20,Assault,D42,130,Milliken,43.835884,-79.254334,2005


In [65]:
# change month in integer to perform great visual
val = {'January': 1,'February': 2,'March': 3,'April':4,'May':5,'June': 6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
mci_2014_2018.replace({"occurrencemonth": val},inplace=True)
# drop duplicate 
mci_2014_2018.drop_duplicates(keep=False,inplace=True) 
#rename Hoo_Id to district_code before merging
mci_2014_2018.rename(columns = {'Hood_ID':'district_code'},inplace = True)
# select features to perform analysis
selected_features = ['occurrenceyear','occurrencemonth','MCI','district_code','X','Y','offence']
mci_2014_2018 = mci_2014_2018[selected_features]

In [68]:
display(mci_2014_2018.info())
display(mci_2014_2018.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167525 entries, 0 to 167524
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   occurrenceyear   167476 non-null  float64
 1   occurrencemonth  167476 non-null  float64
 2   MCI              167525 non-null  object 
 3   district_code    167525 non-null  int64  
 4   X                167525 non-null  float64
 5   Y                167525 non-null  float64
 6   offence          167525 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 10.2+ MB


None

,occurrenceyear,occurrencemonth,MCI,district_code,X,Y,offence
0,2014.0,6.0,Theft Over,76,-79.385193,43.659229,Theft Over
1,2014.0,7.0,Assault,36,-79.425400,43.777592,Pointing A Firearm
2,2014.0,7.0,Robbery,36,-79.425400,43.777592,Robbery With Weapon
3,2014.0,7.0,Break and Enter,132,-79.210373,43.801727,B&E
4,2014.0,7.0,Assault,130,-79.254334,43.835884,Assault


In [92]:
average_mci_per_month = mci_2014_2018.groupby(['district_code','occurrenceyear','occurrencemonth']).MCI.count().to_frame().reset_index().groupby(['district_code']).MCI.mean().to_frame().reset_index()
average_mci_per_month.head()

,district_code,MCI
0,1,61.957143
1,2,30.917808
2,3,10.032787
3,4,11.377049
4,5,9.370968


In [93]:
average_house_sale_prices = house_data.groupby(['district_code']).final_price_10k.mean().to_frame().reset_index()
average_house_sale_prices.head()

,district_code,final_price_10k
0,1,45.395788
1,2,41.670900
2,3,81.185714
3,4,82.369829
4,5,51.688889


In [94]:
incomes_neighbourdhood = house_data[['district_code','mean_district_income_10k']].groupby(['district_code']).mean_district_income_10k.max().to_frame().reset_index()
toronto_neighbourhood_data = toronto_grouped.merge(incomes_neighbourdhood,how='left',on='district_code')

In [95]:
toronto_neighbourhood_data_final=toronto_neighbourhood_data.merge(average_mci_per_month,how ='left',on='district_code')
toronto_neighbourhood_data_final.head()

,district_code,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store